In [1]:
import re
import json
import pandas as pd

In [2]:
EXTRACT_DATE = '21-07-2020'

In [3]:
with open("Lærkeskellet.txt","r",encoding='utf8') as f:
    body = f.read()
    print(body)


Salgspriser - alle boliger - Lærkeskellet, 2000

Adresse                                         Type           Aktuel
                                                               m²
Lærkeskellet 18, 2000 Frederiksberg             Ejerlejlighed
                                                4.575.000      140
Fri handel     15-05-2018                       1.026.500      138

Fri handel     27-07-1994                       Ejerlejlighed  140
                                                4.800.000      140
Lærkeskellet 6, 2000 Frederiksberg              3.070.000
                                                907.133        140
Fri handel     16-11-2017                                      140
                                                Ejerlejlighed  140
Fri handel     31-01-2012                       3.495.000      138
                                                               138
Fri handel     18-11-1994                       Ejerlejlighed
                             

In [4]:
addresses = re.findall('Lærkeskellet \d{1,2},', body)
square_meters = re.findall(' \d{2,3}\n', body)
dates = re.findall('\d\d-\d\d-\d\d\d\d', body)
prices = re.findall('[\d{1,3}.]*\d{3}.\d{3}', body)

The pdf includes information on when the data was downlaoded from Boligsiden. This needs to be removed

I need to be able to find the addresses first and then find the dates for those addesses. Otherwise I can't line up the address, salesdate. 

The salesdate and price have the exact same number of rows, so those already lign up. But one or mole salesdates and prices can be part of the same address.

In [12]:
streetnames = [
    'Ahornskellet',
    'Askeskellet',
    'Birkeskellet',
    'Bøgeskellet',
    'Egeskellet',
    'Lindeskellet',
    'Lærkeskellet',
    'Pileskellet',
    'Plantanskellet',
    'Poppelskellet',
    'Tjørneskellet'
]
rows = []
for street in streetnames:
    with open(f"{street}.txt","r", encoding='utf8') as f:
        body = f.read()
    addresses = re.findall(f'{street} ' + '\d{1,3}, [0-9a-z., ]*2000', body)
    address_chuncks = re.split(f'{street} '+ '\d{1,2},', body)

    prices = re.findall('[\d{1,3}.]*\d{3}.\d{3}', body)
    square_meters = re.findall(' \d{2,3}\n', body)
    
    price_cnt = 0
    for address, m2, text in zip(addresses, square_meters, address_chuncks[1:]):
        m2 = int(m2.replace('\n',''))
        dates = re.findall('\d\d-\d\d-\d\d\d\d', text)
        sale_types = re.findall('Familiehandel|Fri handel|I øvrigt|Auktion', text)
        try:
            dates.remove(EXTRACT_DATE)
        except ValueError:
            pass 

        for (date, sale_type) in zip(dates, sale_types):
            try:
                price = prices[price_cnt]
                price = int(price.replace('.',''))
                
            except:
                break

            m2_price = round(price/m2)
            street_number = address.replace(', 2000','').replace(street, '').replace(' ','').replace(',',' ').replace('.','')
            
            rows.append(
                [
                    street,
                    street_number,
                    date,
                    price,
                    m2,
                    m2_price,
                    sale_type
                ]
            )
            price_cnt += 1

df = pd.DataFrame(columns=[
        'streetName',
        'streetNumber',
        'salesDate',
        'price',
        'm2',
        'm2Price',
        'saleType'
    ],
    data=rows)
len(df)

301

In [16]:
df[
    (df.streetName == "Askeskellet") &
    (df.streetNumber == "14 2")
]

,streetName,streetNumber,salesDate,price,m2,m2Price,saleType
89,Askeskellet,14 2,25-11-2013,956250,111,8615,I øvrigt
90,Askeskellet,14 2,31-01-2000,1610000,111,14505,Fri handel


We will keep only "fri handel" for now. 

In [22]:
df[df.saleType=="Fri handel"].to_csv('../../data/real-estate/sales.csv')

In [18]:
addressesJson = {}
streetJsons = []
for street in df.streetName.unique():
    street_df = df[df.streetName==street]
    streetJson = {
        'street':street,
        'numbers': list(street_df.streetNumber.unique())
    }
    streetJsons.append(streetJson)
addressesJson['streets'] = streetJsons 

In [19]:
with open('../../data/real-estate/addresses.json', 'w', encoding='utf8') as json_file:
  json.dump(addressesJson, json_file)


In [20]:
pd.DataFrame(
    df.m2.unique(),
    columns=['m2']
).to_csv('../../data/real-estate/squaremeters.csv')